In [1]:
file_id='test_data.csv'
df_read=pd.read_csv(file_id)
df_read.columns=df_read.columns.str.lower().values
len(df_read)

1001

In [2]:
types_found=pd.unique(df_read['type'])
types_found

array(['r', 'heading', 'a', 'm'], dtype=object)

In [3]:
imu_sensor_types=['a','m','r']
imu_found=set(types_found).intersection(set(imu_sensor_types))

df_imu=df_read.ix[(df_read['type']=='a')|(df_read['type']=='m')|(df_read['type']=='r'),:]
df_imu.reset_index(inplace=True,drop=True)
df_imu['time']=(df_imu.timestamp-df_imu.timestamp.min())/1e6

f_s=200
time_uniform=np.linspace(df_imu.timestamp.min(), df_imu.timestamp.max(), np.ceil(f_s*df_imu.time.max()))

from scipy.interpolate import interp1d
from scipy import array
# Extrapolation code from: http://stackoverflow.com/questions/2745329/how-to-make-scipy-interpolate-give-an-extrapolated-result-beyond-the-input-range

def extrap1d(interpolator):
    xs = interpolator.x
    ys = interpolator.y

    def pointwise(x):
        if x < xs[0]:
            return ys[0]+(x-xs[0])*(ys[1]-ys[0])/(xs[1]-xs[0])
        elif x > xs[-1]:
            return ys[-1]+(x-xs[-1])*(ys[-1]-ys[-2])/(xs[-1]-xs[-2])
        else:
            return interpolator(x)

    def ufunclike(xs):
        return array(map(pointwise, array(xs)))

    return ufunclike

def extrapolate_1d(time_in,values_in,time_uniform):
    f_x = extrap1d(interp1d(time_in,values_in))
    values_out =f_x(time_uniform)
    return values_out



# time_vec
# df_a=df_imu.ix[(df_imu['type']=='a'),['timestamp','x','y','z']]
# df_a.reset_index(inplace=True,drop=True)
# ts_min=df_imu.timestamp.min()
# ts_max=df_imu.timestamp.max()
# print(ts_min)
# print(ts_max)
# df_a
# # if(len()):
#     x=1
# df_

# poly = np.polyfit(x[:, i], y[:, i], deg=3)
# y_int  = np.polyval(poly, x_val)


C:\toolkits\anaconda2-4.1.0\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
df_imu_out=pd.DataFrame(data=time_uniform,columns=['timestamp'])
imu_srt=sorted(list(imu_found))
sens_present=''.join(map(str, imu_srt))
df_imu_out['sens_present']=sens_present
import itertools
spatial_axes=list(['x','y','z'])
imu_axes_comb=list(itertools.product(imu_found,spatial_axes))
imu_axes_comb
for sens_ax in imu_axes_comb:
    time_in=df_imu.ix[df_imu.type==sens_ax[0],'timestamp']
    values_in=df_imu.ix[df_imu.type==sens_ax[0],sens_ax[1]]
    df_imu_out[sens_ax[0]+sens_ax[1]]=extrapolate_1d(time_in,values_in,time_uniform)
    


In [5]:
df_imu_out

,timestamp,sens_present,ax,ay,az,rx,ry,rz,mx,my,mz
0,1.380054e+09,amr,-0.006073,0.004948,-1.004369,-0.020386,0.020549,0.001251,-29.296599,177.540058,-171.808550
1,1.380059e+09,amr,-0.006042,0.003758,-1.001556,-0.019240,0.020526,0.002425,-29.263641,177.241402,-171.776276
2,1.380064e+09,amr,-0.006011,0.002568,-0.998742,-0.018094,0.020504,0.003599,-29.230683,176.942746,-171.744002
3,1.380069e+09,amr,-0.005988,0.001576,-0.995782,-0.016415,0.019978,0.002634,-29.197725,176.644090,-171.711727
4,1.380074e+09,amr,-0.006119,0.004541,-0.989914,-0.014680,0.019398,0.001439,-29.164768,176.345434,-171.679453
5,1.380079e+09,amr,-0.006128,0.007061,-0.984527,-0.015733,0.021123,0.003112,-29.131810,176.046778,-171.647179
6,1.380084e+09,amr,-0.004427,0.003335,-0.985895,-0.017460,0.023404,0.005477,-29.098852,175.748122,-171.614905
7,1.380089e+09,amr,-0.002873,0.000009,-0.987069,-0.018822,0.019641,0.005372,-29.065894,175.449466,-171.582631
8,1.380094e+09,amr,-0.002757,0.000603,-0.986336,-0.020035,0.013391,0.004252,-29.032936,175.150810,-171.550357
9,1.380099e+09,amr,-0.002828,0.001011,-0.985631,-0.021212,0.010970,0.003822,-28.999979,174.852154,-171.518082


In [6]:

# # values_out = extrapolator( time_out )

In [8]:
%matplotlib inline


In [9]:
# for combos in imu_axes_comb:
#     print(combos[1])